In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_json("News_Category_Dataset_v3.json", encoding="utf8",lines=True)

In [3]:
df

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.6+ MB


In [5]:
import gensim
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api


In [6]:
w2v = api.load("glove-wiki-gigaword-100")

In [7]:
df.headline

0         Over 4 Million Americans Roll Up Sleeves For O...
1         American Airlines Flyer Charged, Banned For Li...
2         23 Of The Funniest Tweets About Cats And Dogs ...
3         The Funniest Tweets From Parents This Week (Se...
4         Woman Who Called Cops On Black Bird-Watcher Lo...
                                ...                        
209522    RIM CEO Thorsten Heins' 'Significant' Plans Fo...
209523    Maria Sharapova Stunned By Victoria Azarenka I...
209524    Giants Over Patriots, Jets Over Colts Among  M...
209525    Aldon Smith Arrested: 49ers Linebacker Busted ...
209526    Dwight Howard Rips Teammates After Magic Loss ...
Name: headline, Length: 209527, dtype: object

In [8]:
import numpy as np
import re

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text.split()

def sentence_to_avg_vector(sentence, model, dim=100):
    words = preprocess(sentence)
    vectors = [model[word] for word in words if word in model]
    if not vectors:
        return np.zeros(dim)
    return np.mean(vectors, axis=0)


In [29]:
from sklearn.metrics.pairwise import cosine_similarity

def vector_similarity(vec1, vec2, model): 
    return cosine_similarity([vec1], [vec2])[0][0]


In [27]:
sentence_vectors = [sentence_to_avg_vector(t, w2v) for t in df.headline]

In [30]:
text = "100 men killed a gorilla"
text_vector = sentence_to_avg_vector(text,w2v)
similarity_score = -2
for vec in sentence_vectors:
    similarity_score = max(vector_similarity(vec, text_vector,w2v),similarity_score)
print("Plagarism Score: ",similarity_score)    

Plagarism Score:  0.886264
